<a href="https://colab.research.google.com/github/jasongu927/vision/blob/master/gans2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#can tensorboard work!
!pip install -q tf-nightly-2.0-preview
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


device_name = tf.test.gpu_device_name()
USE_GPU = True
if device_name != '/device:GPU:0':
  USE_GPU = False
  #raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


#mnist=input_data.read_data_sets("MNIST_data")

Found GPU at: 


In [0]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
#from tensorflow.keras import layers
import time

import tensorflow as tf
tfgan = tf.contrib.gan
framework = tf.contrib.framework

queues = tf.contrib.slim.queues
layers = tf.contrib.layers
ds = tf.contrib.distributions
leaky_relu = lambda net: tf.nn.leaky_relu(net, alpha=0.01)




from IPython import display

AttributeError: ignored

In [0]:
def generator_model_fn(z): 
    z, conditioning = z
    #with tf.variable_scope('generator', reuse=tf.AUTO_REUSE):

    net = tf.keras.layers.Dense(1024)(z)
    #net = tf.keras.layers.BatchNormalization(momentum=0.9)(net)
    net = tf.keras.layers.ReLU()(net)
    net = tfgan.features.condition_tensor_from_onehot(net, conditioning)
    net = tf.keras.layers.Dense(7*7*256)(net)
    #net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.ReLU()(net)

    net = tf.keras.layers.Reshape((7, 7, 256))(net)

    net = tf.keras.layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=True)(net)
    #net = tf.keras.layers.BatchNormalization()(net)
    #net = tf.keras.layers.LeakyReLU()(net)


    net = tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=True)(net)
    net = tf.keras.layers.BatchNormalization()(net)
    net = tf.keras.layers.LeakyReLU()(net)

    net = tf.keras.layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(net)

    return net

In [0]:
def discriminator_model_fn(x, unused_conditioning):
    #with tf.variable_scope('discriminator', reuse=tf.AUTO_REUSE):

    
    net = tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same')(x)
    #net = tf.keras.layers.BatchNormalization(momentum=0.9)(net)
    net = tf.keras.layers.LeakyReLU()(net)

    net = tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same')(net)
    
    net = tf.keras.layers.Flatten()(net)
    net = tf.keras.layers.Dense(256)(net)
    net = tf.keras.layers.LeakyReLU()(net)
    net = tf.keras.layers.BatchNormalization(momentum=0.9)(net)
    net = tf.keras.layers.Dropout(0.3)(net)
    net = tf.keras.layers.Dense(1)(net)
    """
    input_layer = tf.reshape(x, [-1, 28, 28, 1])
    net = tf.keras.layers.Conv2D(64, (3, 3), name="conv1", strides=(2,2))(input_layer)
    net = tf.keras.layers.Conv2D(64, (3, 3), name="conv2", strides=(2,2))(net)
    net = tf.keras.layers.MaxPooling2D((2,2), 1)(net)
    net = tf.keras.layers.Flatten()(net)
    net = tf.keras.layers.Dense(128*2, activation=tf.nn.relu6)(net)
    net = tf.keras.layers.Dropout(0.25)(net)
    net = tf.keras.layers.Dense(units=1)(net)"""
    return net

In [0]:
def generator_fn(feat, weight_decay=2.5e-5, is_training=True):
    """Simple generator to produce MNIST images.
    
    Args:
        noise: A single Tensor representing noise.
        weight_decay: The value of the l2 weight decay.
        is_training: If `True`, batch norm uses batch statistics. If `False`, batch
            norm uses the exponential moving average collected from population 
            statistics.
    
    Returns:
        A generated image in the range [-1, 1].
    """
    #noise , one_hot_labels = feat
    noise = feat[0]
    one_hot_labels = feat[1]
    with tf.device('/gpu:0'):
      with framework.arg_scope(
        [layers.fully_connected, layers.conv2d_transpose],
        activation_fn=tf.nn.relu, normalizer_fn=layers.batch_norm,
        weights_regularizer=layers.l2_regularizer(weight_decay)),\
        framework.arg_scope([layers.batch_norm], is_training=is_training,
                        zero_debias_moving_mean=True):
        net = layers.fully_connected(noise, 1024)
        net = tfgan.features.condition_tensor_from_onehot(net, one_hot_labels)
        net = layers.fully_connected(net, 7 * 7 * 256)
        net = tf.reshape(net, [-1, 7, 7, 256])
        #net = layers.conv2d_transpose(net, 256, [5, 5], stride=1)
        net = layers.conv2d_transpose(net, 128, [5, 5], stride=2)
        net = layers.conv2d_transpose(net, 64, [5, 5], stride=2)
        # Make sure that generator output is in the same range as `inputs`
        # ie [-1, 1].
        net = layers.conv2d(net, 1, 4, normalizer_fn=None, activation_fn=tf.tanh)

        return net

In [0]:
def discriminator_fn(img, conditioning, weight_decay=2.5e-5,
                     is_training=True):
    """Discriminator network on MNIST digits.
    
    Args:
        img: Real or generated MNIST digits. Should be in the range [-1, 1].
        unused_conditioning: The TFGAN API can help with conditional GANs, which
            would require extra `condition` information to both the generator and the
            discriminator. Since this example is not conditional, we do not use this
            argument.
        weight_decay: The L2 weight decay.
        is_training: If `True`, batch norm uses batch statistics. If `False`, batch
            norm uses the exponential moving average collected from population 
            statistics.
    
    Returns:
        Logits for the probability that the image is real.
    """
    _, one_hot_labels = conditioning
    with tf.device('/gpu:0'):    
      with framework.arg_scope(
        [layers.conv2d, layers.fully_connected],
        activation_fn=leaky_relu, normalizer_fn=None,
        weights_regularizer=layers.l2_regularizer(weight_decay),
        biases_regularizer=layers.l2_regularizer(weight_decay)):
        net = layers.conv2d(img, 64, [5, 5], stride=2)
        net = tfgan.features.condition_tensor_from_onehot(net, one_hot_labels)
        net = layers.conv2d(net, 128, [5, 5], stride=2)
        #net = layers.conv2d(net, 256, [5, 5], stride=1)
        net = layers.flatten(net)
        with framework.arg_scope([layers.batch_norm], is_training=is_training):
            net = layers.fully_connected(net, 1024, normalizer_fn=layers.batch_norm)
            net = tf.keras.layers.Dropout(0.25)(net)
        return layers.linear(net, 1)

In [0]:
import tensorflow.keras as keras 

((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()
print(train_data.shape)

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required
train_data_1 = train_data.reshape(train_data.shape[0], 28, 28, 1)
eval_data_1 = eval_data.reshape(eval_data.shape[0], 28, 28, 1)

train_labels_1 = keras.utils.to_categorical(train_labels, 10)
eval_labels_1 = keras.utils.to_categorical(eval_labels, 10)

Using Estimator API

In [0]:
#tf.reset_default_graph()

def _get_train_input_fn(batch_size, noise_dims):
    def train_input_fn(): #try with tf.data
        real_images, labels = tf.train.batch((train_data_1, train_labels_1), batch_size=batch_size, enqueue_many=True)
        noise = tf.random_normal([batch_size, noise_dims])
        #return noise, real_images
        return ((noise, labels), real_images)
    return train_input_fn


def _get_predict_input_fn(batch_size, noise_dims):
    def predict_input_fn():
        noise = tf.random_normal([batch_size, noise_dims])
        one_hots = [x%10 for x in range(batch_size)]
        return ((noise, tf.one_hot(one_hots,10)),None)
        #return noise
    return predict_input_fn

In [0]:
BATCH_SIZE = 16#@param
NOISE_DIMS = 100#@param
NUM_STEPS = 60000/BATCH_SIZE#@param
NUM_EPOCHS = 5#@param
GEN_LEARNING_RATE = 0.00002 #@param
DISC_LEARNING_RATE = 0.00006 #@param

print(train_data_1.shape)

# Initialize GANEstimator with options and hyperparameters.
gan_estimator = tfgan.estimator.GANEstimator(
    generator_fn=generator_model_fn,
    #discriminator_fn=discriminator_model_fn,
    #generator_fn=generator_fn,
    discriminator_fn=discriminator_fn,
    model_dir = "tmp/tbr",
    generator_loss_fn=tfgan.losses.wasserstein_generator_loss, 
    discriminator_loss_fn=tfgan.losses.wasserstein_discriminator_loss,
    #generator_loss_fn=tfgan.losses.modified_generator_loss, 
    #discriminator_loss_fn=tfgan.losses.modified_discriminator_loss,
    generator_optimizer=tf.train.AdamOptimizer(GEN_LEARNING_RATE, 0.5),
    discriminator_optimizer=tf.train.AdamOptimizer(DISC_LEARNING_RATE, 0.5),
    config =tf.estimator.RunConfig(log_step_count_steps=NUM_STEPS/3))

In [0]:

# Train estimator.
train_input_fn = _get_train_input_fn(BATCH_SIZE, NOISE_DIMS)
start_time = time.time()
gan_estimator.train(train_input_fn, steps = 1)
time_since_start = (time.time() - start_time) / 60.0
print('Time since start: %f m' % time_since_start)
print('Steps per min: %f' % (NUM_STEPS / time_since_start))

In [0]:
from six.moves import xrange  # pylint: disable=redefined-builtin

def _get_next(iterable):
    try:
        return iterable.next()  # Python 2.x.x
    except AttributeError:
        return iterable.__next__()  # Python 3.x.x


# Run inference.
predict_input_fn = _get_predict_input_fn(16, NOISE_DIMS)
prediction_iterable = gan_estimator.predict(
    predict_input_fn, hooks=[tf.train.StopAtStepHook(last_step=1)])
predictions = list(prediction_iterable)

try: # Close the predict session.
    _get_next(prediction_iterable)
except StopIteration:
    pass

# Nicely tile output and visualize.
for i in range(1,17):
  plt.subplot(4,4,i)
  plt.imshow(predictions[i-1].reshape(28,28))
plt.show()

      

for i in range(NUM_EPOCHS):
  start_time = time.time()
  gan_estimator.train(train_input_fn, steps = NUM_STEPS)
  time_since_start = (time.time() - start_time) / 60.0
  print("~"*40)
  print("EPOCH %d" % i)
  print("~"*40)
  print('Time since start: %f m' % time_since_start)
  print('Steps per min: %f' % (NUM_STEPS / time_since_start))

  # Run inference.
  predict_input_fn = _get_predict_input_fn(16, NOISE_DIMS)
  prediction_iterable = gan_estimator.predict(
      predict_input_fn)
  predictions = [_get_next(prediction_iterable) for _ in xrange(16)]

  try: # Close the predict session.
      _get_next(prediction_iterable)
  except StopIteration:
      pass

  # Nicely tile output and visualize.
  for i in range(1,17):
    plt.subplot(4,4,i)
    plt.imshow(predictions[i-1].reshape(28,28))
  plt.show()


In [0]:
%tensorboard --logdir tmp/tbr

# Cats #3
from https://github.com/utkd/gans/blob/master/cifar10dcgan.ipynb

In [0]:
from __future__ import print_function, division

from keras.layers import Input, Dense, Flatten, Dropout, Reshape
from keras.layers import BatchNormalization, Activation, Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras.optimizers import Adam

from keras.datasets import cifar10
import keras.backend as K

import matplotlib.pyplot as plt

import sys
import numpy as np

%pylab inline


In [0]:
def get_generator(input_layer):
  '''
  Requires the input layer as input, outputs the model and the final layer
  '''
  
  hid = Dense(128 * 16 * 16, activation='relu')(input_layer)    
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
  hid = Reshape((16, 16, 128))(hid)

  hid = Conv2D(128, kernel_size=5, strides=1,padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)    
  #hid = Dropout(0.5)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2DTranspose(128, 4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  #hid = Dropout(0.5)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)
                      
  hid = Conv2D(3, kernel_size=5, strides=1, padding="same")(hid)
  out = Activation("tanh")(hid)

  model = Model(input_layer, out)
  model.summary()
  
  return model, out

In [0]:

def get_discriminator(input_layer):
  '''
  Requires the input layer as input, outputs the model and the final layer
  '''

  hid = Conv2D(128, kernel_size=3, strides=1, padding='same')(input_layer)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
  hid = BatchNormalization(momentum=0.9)(hid)
  hid = LeakyReLU(alpha=0.1)(hid)

  hid = Flatten()(hid)
  hid = Dropout(0.4)(hid)
  hid = Dense(128)(hid)
  hid = Dropout(0.2)(hid)
  out = Dense(1, activation='sigmoid')(hid)

  model = Model(input_layer, out)

  model.summary()

  return model, out

In [0]:
from keras.preprocessing import image

def generate_noise(n_samples, noise_dim):
  X = np.random.normal(0, 1, size=(n_samples, noise_dim))
  return X

def show_imgs(batchidx):
  noise = generate_noise(9, 100)
  gen_imgs = generator.predict(noise)
  plt.figure(figsize=(10,10))
  fig, axs = plt.subplots(3, 3)
  count = 0
  for i in range(3):
    for j in range(3):
      # Dont scale the images back, let keras handle it
      img = image.array_to_img(gen_imgs[count], scale=True)
      axs[i,j].imshow(img)
      axs[i,j].axis('off')
      count += 1
  plt.show()
  plt.close()

In [0]:
# GAN creation
img_input = Input(shape=(32,32,3))
discriminator, disc_out = get_discriminator(img_input)
discriminator.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy', metrics=['accuracy'])

discriminator.trainable = False

noise_input = Input(shape=(100,))
generator, gen_out = get_generator(noise_input)

gan_input = Input(shape=(100,))
x = generator(gan_input)
gan_out = discriminator(x)
gan = Model(gan_input, gan_out)
gan.summary()

gan.compile(optimizer=Adam(0.0002, 0.5), loss='binary_crossentropy')

In [0]:
BATCH_SIZE = 16

# # Get training images
(X_train, y_train), (X_test, _) = cifar10.load_data()

# Select Cats
X_train = X_train[y_train[:,0]==3]
print ("Training shape: {}".format(X_train.shape))

# Normalize data
X_train = (X_train - 127.5) / 127.5
 
num_batches = int(X_train.shape[0]/BATCH_SIZE)

In [0]:

N_EPOCHS = 100 #@param
for epoch in range(N_EPOCHS):

  cum_d_loss = 0.
  cum_g_loss = 0.
  
  for batch_idx in range(num_batches):
    # Get the next set of real images to be used in this iteration
    images = X_train[batch_idx*BATCH_SIZE : (batch_idx+1)*BATCH_SIZE]

    noise_data = generate_noise(BATCH_SIZE, 100)
    generated_images = generator.predict(noise_data)

    # Train on soft labels (add noise to labels as well)
    noise_prop = 0.05 # Randomly flip 5% of labels
    
    # Prepare labels for real data
    true_labels = np.zeros((BATCH_SIZE, 1)) + np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
    flipped_idx = np.random.choice(np.arange(len(true_labels)), size=int(noise_prop*len(true_labels)))
    true_labels[flipped_idx] = 1 - true_labels[flipped_idx]
    
    # Train discriminator on real data
    d_loss_true = discriminator.train_on_batch(images, true_labels)

    # Prepare labels for generated data
    gene_labels = np.ones((BATCH_SIZE, 1)) - np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
    flipped_idx = np.random.choice(np.arange(len(gene_labels)), size=int(noise_prop*len(gene_labels)))
    gene_labels[flipped_idx] = 1 - gene_labels[flipped_idx]
    
    # Train discriminator on generated data
    d_loss_gene = discriminator.train_on_batch(generated_images, gene_labels)

    d_loss = 0.5 * np.add(d_loss_true, d_loss_gene)
    cum_d_loss += d_loss

    # Train generator
    noise_data = generate_noise(BATCH_SIZE, 100)
    g_loss = gan.train_on_batch(noise_data, np.zeros((BATCH_SIZE, 1)))
    cum_g_loss += g_loss

  print('  Epoch: {}, Generator Loss: {}, Discriminator Loss: {}'.format(epoch+1, cum_g_loss/num_batches, cum_d_loss/num_batches))
  show_imgs("epoch" + str(epoch))